In [1]:
# Imports

import os
import string
import re
from collections import Counter
from cltk.corpus.readers import get_corpus_reader


from cltk.tokenize.sentence import TokenizeSentence
from cltk.tokenize.word import WordTokenizer
from cltk.lemmatize.latin.backoff import BackoffLatinLemmatizer
from cltk.stem.latin.j_v import JVReplacer
from cltk.utils.file_operations import open_pickle

In [2]:
# Load CLTK Tesserae corpus

tess = get_corpus_reader(corpus_name = 'latin_text_tesserae', language = 'latin')

In [3]:
# Setup CLTK tools

word_tokenizer = WordTokenizer('latin')
sent_tokenizer = TokenizeSentence('latin')
lemmatizer = BackoffLatinLemmatizer()
replacer = JVReplacer()

In [4]:
# Get list of texts

tess_texts = [text for text in tess.texts(tess.fileids())]

In [5]:
# Preprocessing script

def preprocess(text):    
    
    text = text.lower()

    punctuation = string.punctuation
    punctuation += "\"#$%&\'()*+,-/:;<=>@[\]^_`{|}~"
    translator = str.maketrans({key: " " for key in punctuation})
    text = text.translate(translator)
    
    translator = str.maketrans({key: " " for key in '0123456789'})
    text = text.translate(translator)
    
    text = replacer.replace(text)
    
    text = " ".join(text.split('\n'))
    text = " ".join(text.split())
    
    return text

In [6]:
# Preprocess texts

tess_pp = [preprocess(text).split() for text in tess_texts]

In [7]:
# Flatten texts

tess_flat = " ".join([item for sublist in tess_pp for item in sublist])

In [8]:
# Tokenize texts

tess_tokens = word_tokenizer.tokenize(tess_flat)
print(tess_tokens[:10])

['galli', 'caesaris', 'saeuitia', 'post', 'emensos', 'insuperabilis', 'expeditionis', 'euentus', 'languentibus', 'partium']


In [9]:
# Get total token counts

tess_tokens_len = len(tess_tokens)
tess_tokens_set_len = len(set(tess_tokens))

In [10]:
# Print top 10 token counts

print('Number of tokens in CLTK Tesserae:', tess_tokens_len)
print('Number of unique tokens in CLTK Tesserae:', tess_tokens_set_len)

Number of tokens in CLTK Tesserae: 6864495
Number of unique tokens in CLTK Tesserae: 310930


In [11]:
# Build counter of top token counts

tess_tokens_counter = Counter(tess_tokens)
tess_tokens_mc = tess_tokens_counter.most_common(10000)

running = 0

print('Top 25 tokens in CLTK Tesserae:\n')
print("{number:>5}  {token:<12}{count:<12}{percent:<12}{running:<12}".format(number="", token="TOKEN", count="COUNT", percent="Type-Tok %", running = "RUNNING %"))
for i, pair in enumerate(tess_tokens_mc[:10]):
    running += pair[1]
    print("{number:>5}. {token:<12}{count:<12}{percent:<12}{running:<12}".format(number=i+1, token=pair[0], count=pair[1], percent=str(round(pair[1] / len(tess_tokens)*100, 2))+"%", running = str(round(running / len(tess_tokens)*100, 2))+"%"))

Top 25 tokens in CLTK Tesserae:

       TOKEN       COUNT       Type-Tok %  RUNNING %   
    1. et          225907      3.29%       3.29%       
    2. in          141415      2.06%       5.35%       
    3. est         81715       1.19%       6.54%       
    4. non         77337       1.13%       7.67%       
    5. ut          60016       0.87%       8.54%       
    6. ad          55140       0.8%        9.35%       
    7. cum         53193       0.77%       10.12%      
    8. quod        44106       0.64%       10.76%      
    9. qui         43352       0.63%       11.39%      
   10. si          38532       0.56%       11.96%      


In [12]:
# Lemmatize texts

tess_lemma_pairs = lemmatizer.lemmatize(tess_tokens)
print(tess_lemma_pairs[:10])

[('galli', 'gallus'), ('caesaris', 'caesar'), ('saeuitia', 'saevitia'), ('post', 'post'), ('emensos', 'emetior'), ('insuperabilis', 'insuperabilis'), ('expeditionis', 'expeditio'), ('euentus', 'euentus'), ('languentibus', 'langueo'), ('partium', 'pars')]


In [13]:
# Get total lemma counts

tess_lemmas = [lemma[1] for lemma in tess_lemma_pairs]
tess_lemmas_set_len = len(set(tess_lemmas))

In [14]:
# Print top 10 token counts

print('Number of tokens in CLTK Tesserae:', tess_tokens_len)
print('Number of unique tokens in CLTK Tesserae:', tess_tokens_set_len)
print('Number of unique lemmas in CLTK Tesserae:', tess_lemmas_set_len)

Number of tokens in CLTK Tesserae: 6864495
Number of unique tokens in CLTK Tesserae: 310930
Number of unique lemmas in CLTK Tesserae: 143334


In [15]:
# Build counter of top lemma counts

tess_lemmas_counter = Counter(tess_lemmas)
tess_lemmas_mc = tess_lemmas_counter.most_common(10000)

running = 0

print('Top 10 lemmas in Latin Library:\n')
print("{number:>5}  {lemma:<12}{count:<12}{percent:<12}{running:<12}".format(number="", lemma="LEMMA", count="COUNT", percent="TYPE-LEM %", running = "RUNNING %"))
for i, pair in enumerate(tess_lemmas_mc[:10]):
    running += pair[1]
    print("{number:>5}. {lemma:<12}{count:<12}{percent:<12}{running:<12}".format(number=i+1, lemma=pair[0], count=pair[1], percent=str(round(pair[1] / len(tess_tokens)*100, 2))+"%", running = str(round(running / len(tess_tokens)*100, 2))+"%"))    

Top 10 lemmas in Latin Library:

       LEMMA       COUNT       TYPE-LEM %  RUNNING %   
    1. et          225907      3.29%       3.29%       
    2. sum         214791      3.13%       6.42%       
    3. qui         175443      2.56%       8.98%       
    4. in          141958      2.07%       11.04%      
    5. is          98416       1.43%       12.48%      
    6. non         77337       1.13%       13.6%       
    7. hic         74066       1.08%       14.68%      
    8. ut          60630       0.88%       15.57%      
    9. ad          55145       0.8%        16.37%      
   10. cum2        53693       0.78%       17.15%      


In [16]:
# Print top 10,000 counts

print('Top 10,000 tokens in the CLTK Tesserae:\n')
for i, pair in enumerate(tess_tokens_mc):
    print("{number}. {token} ({count})".format(number=i+1, token=pair[0], count=pair[1]))

Top 10,000 tokens in the CLTK Tesserae:

1. et (225907)
2. in (141415)
3. est (81715)
4. non (77337)
5. ut (60016)
6. ad (55140)
7. cum (53193)
8. quod (44106)
9. qui (43352)
10. si (38532)
11. sed (35860)
12. quae (35637)
13. a (33616)
14. de (32082)
15. quam (30763)
16. ex (26660)
17. esse (25889)
18. nec (24971)
19. aut (23463)
20. me (21893)
21. sunt (21577)
22. quid (20852)
23. per (20593)
24. hoc (20467)
25. atque (20390)
26. enim (19874)
27. se (19803)
28. te (19672)
29. etiam (18437)
30. autem (18139)
31. ab (17907)
32. eius (16720)
33. ne (16650)
34. ac (16342)
35. quo (15744)
36. haec (15324)
37. mihi (14960)
38. iam (14957)
39. sit (13517)
40. id (13383)
41. tamen (13305)
42. tibi (12554)
43. e (12282)
44. ego (12219)
45. neque (11872)
46. nam (11692)
47. quidem (11543)
48. quoque (11417)
49. ita (11375)
50. uel (11229)
51. nunc (10655)
52. eo (10092)
53. nihil (10047)
54. erat (10013)
55. quia (9968)
56. quem (9655)
57. pro (9644)
58. sic (9552)
59. eum (9544)
60. ea (9538)

981. uocatur (731)
982. posuit (729)
983. constat (729)
984. orbis (728)
985. aquam (728)
986. plerumque (727)
987. fuerint (727)
988. loqui (725)
989. aures (725)
990. magnis (724)
991. iuuenis (723)
992. putant (722)
993. refert (722)
994. usu (722)
995. operam (722)
996. terrarum (719)
997. ferunt (719)
998. uictoria (719)
999. patria (719)
1000. uides (719)
1001. melle (719)
1002. maius (716)
1003. oris (716)
1004. metus (714)
1005. uxor (713)
1006. maiores (713)
1007. acie (713)
1008. petit (713)
1009. militibus (711)
1010. romana (711)
1011. proelia (711)
1012. accepit (711)
1013. nox (710)
1014. matrem (710)
1015. facto (709)
1016. sole (707)
1017. uelit (707)
1018. litora (707)
1019. uestris (707)
1020. utraque (706)
1021. uisum (706)
1022. circum (705)
1023. solo (704)
1024. fecisse (704)
1025. certa (704)
1026. nota (703)
1027. periculum (703)
1028. maiore (702)
1029. iuris (700)
1030. potestatem (700)
1031. dic (700)
1032. quaeso (699)
1033. roma (698)
1034. plane (698)
1035

2532. cupio (296)
2533. coniugis (296)
2534. mensibus (296)
2535. oras (296)
2536. altare (296)
2537. facilis (295)
2538. curia (295)
2539. potuerunt (295)
2540. traditur (295)
2541. uelint (295)
2542. faucibus (295)
2543. dentes (295)
2544. martis (294)
2545. sententiis (294)
2546. hannibalem (294)
2547. officia (294)
2548. uallum (294)
2549. uolucres (294)
2550. pertinere (294)
2551. audita (294)
2552. positis (294)
2553. mulierem (294)
2554. uolet (294)
2555. nescit (294)
2556. det (294)
2557. generum (293)
2558. conscientia (293)
2559. captus (293)
2560. familiaris (293)
2561. ipsas (293)
2562. pedem (293)
2563. totis (293)
2564. fuissent (293)
2565. milite (292)
2566. uetere (292)
2567. ueneno (292)
2568. societatem (292)
2569. noctu (292)
2570. misi (292)
2571. auris (292)
2572. boues (292)
2573. hospes (292)
2574. uates (291)
2575. condita (291)
2576. responsum (291)
2577. signo (291)
2578. ferox (291)
2579. condicione (291)
2580. mouit (291)
2581. quales (291)
2582. praedae (29

4362. uenturum (174)
4363. pauor (174)
4364. leti (174)
4365. mel (174)
4366. peccatorum (174)
4367. uentrem (174)
4368. nonnulli (174)
4369. saluum (174)
4370. ros (174)
4371. intrauit (174)
4372. posuerunt (174)
4373. ph (174)
4374. nobilem (173)
4375. quaestione (173)
4376. criminibus (173)
4377. statione (173)
4378. proficisci (173)
4379. prodidit (173)
4380. reuerentia (173)
4381. laeti (173)
4382. ostia (173)
4383. uenas (173)
4384. iudicibus (173)
4385. homerus (173)
4386. limite (173)
4387. experiri (173)
4388. damno (173)
4389. superi (173)
4390. blanda (173)
4391. sistit (173)
4392. canticum (173)
4393. linea (173)
4394. operi (173)
4395. diligit (173)
4396. actio (173)
4397. area (173)
4398. gressus (172)
4399. fiduciam (172)
4400. militis (172)
4401. consularis (172)
4402. figuras (172)
4403. minis (172)
4404. cladis (172)
4405. ripa (172)
4406. tectum (172)
4407. uersu (172)
4408. parietes (172)
4409. restituit (172)
4410. uetera (172)
4411. bibit (172)
4412. scripturarum 

5861. auia (127)
5862. caedibus (127)
5863. perquam (127)
5864. ingentia (127)
5865. philosophus (127)
5866. crebra (127)
5867. fortitudinem (127)
5868. roboris (127)
5869. praemio (127)
5870. puluerem (127)
5871. legato (127)
5872. breuem (127)
5873. ceruicem (127)
5874. hominumque (127)
5875. utilitate (127)
5876. euadere (127)
5877. uiderent (127)
5878. δὲ (127)
5879. faciendi (127)
5880. bos (127)
5881. effecit (127)
5882. refertur (127)
5883. concessa (127)
5884. comitiorum (127)
5885. quaere (127)
5886. nactus (127)
5887. pateretur (127)
5888. successit (127)
5889. falsis (127)
5890. longam (127)
5891. senectute (127)
5892. xiiii (127)
5893. ueneri (127)
5894. impii (127)
5895. seruant (127)
5896. imperatorum (127)
5897. habuerat (127)
5898. instare (127)
5899. quaenam (127)
5900. iudico (127)
5901. calamitate (127)
5902. ap (127)
5903. seri (127)
5904. brute (127)
5905. damus (127)
5906. samnites (127)
5907. bestiae (126)
5908. ualido (126)
5909. colit (126)
5910. exceptis (126

7861. uesicae (95)
7862. angues (95)
7863. liuius (95)
7864. axis (95)
7865. uirgilius (95)
7866. simon (95)
7867. aguntur (94)
7868. easque (94)
7869. interrogatus (94)
7870. multiplici (94)
7871. sorore (94)
7872. proiecit (94)
7873. participes (94)
7874. priuati (94)
7875. maestus (94)
7876. absolutus (94)
7877. familiares (94)
7878. periculosum (94)
7879. confiteor (94)
7880. extremam (94)
7881. facinoris (94)
7882. mediocris (94)
7883. pansa (94)
7884. exordia (94)
7885. hiatu (94)
7886. theatrum (94)
7887. salutaris (94)
7888. notos (94)
7889. mixtus (94)
7890. stultitiam (94)
7891. laeuum (94)
7892. incubuit (94)
7893. condiciones (94)
7894. conscientiae (94)
7895. interfici (94)
7896. grauibus (94)
7897. contionibus (94)
7898. ceteraque (94)
7899. credentes (94)
7900. omnique (94)
7901. incensum (94)
7902. caloris (94)
7903. transitum (94)
7904. rerumque (94)
7905. dissimulare (94)
7906. imperatum (94)
7907. triennium (94)
7908. stabit (94)
7909. dictatore (94)
7910. medicum (9

9360. uictrix (79)
9361. eripi (79)
9362. uicinus (79)
9363. calculos (79)
9364. flexus (79)
9365. agant (79)
9366. facem (79)
9367. consecuta (79)
9368. decernere (79)
9369. portenta (79)
9370. gloriosum (79)
9371. matrum (79)
9372. omine (79)
9373. oracula (79)
9374. licitum (79)
9375. nuntiauit (79)
9376. audientibus (79)
9377. admonitus (79)
9378. paris (79)
9379. periclis (79)
9380. tremor (79)
9381. nutrit (79)
9382. uallibus (79)
9383. finxit (79)
9384. exorta (79)
9385. festinat (79)
9386. elephantos (79)
9387. transeunt (79)
9388. lucilius (79)
9389. drusus (79)
9390. minucius (79)
9391. personas (79)
9392. meretricem (79)
9393. diligunt (79)
9394. nouorum (79)
9395. uetant (79)
9396. armatum (79)
9397. addi (79)
9398. pertineat (79)
9399. dotem (79)
9400. improbe (79)
9401. heredes (79)
9402. tibia (79)
9403. hospites (79)
9404. crederent (79)
9405. chorus (79)
9406. referebat (79)
9407. exhibuit (79)
9408. exeunt (79)
9409. obseruatione (79)
9410. matutino (79)
9411. rubor (

In [17]:
# Print top 10,000 lemma counts

print('Top 10,000 lemmas in the CLTK Tesserae:\n')
for i, pair in enumerate(tess_lemmas_mc):
    print("{number}. {lemma} ({count})".format(number=i+1, lemma=pair[0], count=pair[1]))

Top 10,000 lemmas in the CLTK Tesserae:

1. et (225907)
2. sum (214791)
3. qui (175443)
4. in (141958)
5. is (98416)
6. non (77337)
7. hic (74066)
8. ut (60630)
9. ad (55145)
10. cum2 (53693)
11. tu (53215)
12. ego (52681)
13. ab (51561)
14. ille (45180)
15. omnis (38813)
16. si (38536)
17. neque (36844)
18. atque (36733)
19. sed (36171)
20. dico (33128)
21. de (32268)
22. suus (31146)
23. quam (30763)
24. facio (30551)
25. sui (30257)
26. possum (29438)
27. ipse (29304)
28. res (27763)
29. quis (27749)
30. ex (26694)
31. aut (23463)
32. magnus (22884)
33. uideo (21923)
34. multus (21737)
35. tuus (21441)
36. habeo (21202)
37. per (20955)
38. meus (20633)
39. enim (19874)
40. do (19708)
41. etiam (18568)
42. alius (18549)
43. autem (18142)
44. deus (17220)
45. ne (16650)
46. iam (15961)
47. quo (15744)
48. idem (15128)
49. nos (14926)
50. homo (14815)
51. uerus (13896)
52. dies (13552)
53. bonus (13525)
54. tamen (13323)
55. unus (13301)
56. dominus (13202)
57. uolo (12907)
58. pars (1

853. uersus (1134)
854. parco (1130)
855. profero (1130)
856. respicio (1127)
857. recens (1126)
858. iaceo (1126)
859. proficio (1124)
860. oleum (1124)
861. w (1124)
862. confiteor (1121)
863. interim (1121)
864. pudor (1118)
865. bibo1 (1118)
866. protinus (1117)
867. arx (1116)
868. vester (1115)
869. verto (1114)
870. ai (1114)
871. puella (1112)
872. insula (1109)
873. exerceo (1104)
874. expono (1103)
875. adicio (1102)
876. opto (1099)
877. mollis (1098)
878. irascor (1097)
879. natus (1097)
880. testimonium (1097)
881. num (1094)
882. divido (1093)
883. rego (1089)
884. doleo (1087)
885. adhibeo (1086)
886. ecclesia (1086)
887. utrum (1084)
888. centum (1083)
889. laboro (1079)
890. etsi (1075)
891. inanis (1074)
892. creber (1066)
893. cohors (1066)
894. nimis (1064)
895. experior (1063)
896. opprimo (1060)
897. misericordia (1060)
898. adulescens (1059)
899. impleo (1058)
900. septem (1058)
901. conor (1057)
902. familiaris (1056)
903. conspectus (1056)
904. cursus (1054)
90

1352. mane (666)
1353. fulmen (665)
1354. tabernaculum (665)
1355. lacus (665)
1356. artus (665)
1357. como1 (664)
1358. propterea (664)
1359. circumdo (663)
1360. desero (663)
1361. turbo (662)
1362. mandatum (662)
1363. umerus (662)
1364. suspendo (661)
1365. lac (661)
1366. capillus (661)
1367. fax (661)
1368. coquo (661)
1369. aedifico (660)
1370. nudus (660)
1371. absolvo (659)
1372. lucus (658)
1373. cruor (657)
1374. maiestas (657)
1375. intereo (657)
1376. incendo (656)
1377. brutus (655)
1378. ala (653)
1379. septimus (653)
1380. pedis (652)
1381. angustus (651)
1382. humilis (651)
1383. triginta (651)
1384. merces (650)
1385. ferox (650)
1386. natio (650)
1387. bini (649)
1388. pinguis (649)
1389. obicio (649)
1390. loco (648)
1391. accusator (647)
1392. diues (647)
1393. perduco (647)
1394. caueo (645)
1395. dedo (643)
1396. monumentum (643)
1397. ulcus (643)
1398. phoebus (643)
1399. impetro (642)
1400. festino (642)
1401. nubes (642)
1402. graecia (642)
1403. queo (642)
14

2352. attollo (350)
2353. obliquus (350)
2354. aula (350)
2355. orbo (350)
2356. infirmitas (350)
2357. parricidium (350)
2358. somnium (349)
2359. effero (349)
2360. disputatio (349)
2361. καὶ (349)
2362. resolvo (349)
2363. dissero2 (349)
2364. repleo (349)
2365. dh (349)
2366. palatium (348)
2367. imitor (348)
2368. vena (348)
2369. immolo (348)
2370. mano (348)
2371. palmo (348)
2372. polio1 (347)
2373. galea (347)
2374. moses (347)
2375. subdo (346)
2376. confugio (346)
2377. obsido (346)
2378. vexo (346)
2379. ferreus (346)
2380. praemitto (346)
2381. gloriosus (345)
2382. stupeo (345)
2383. aquilo (345)
2384. introeo (344)
2385. maledico (344)
2386. declino (343)
2387. avidus (343)
2388. temeritas (343)
2389. mico (343)
2390. vinea (343)
2391. gena (343)
2392. â (343)
2393. gradior (342)
2394. increpo (341)
2395. cottidianus (341)
2396. neuter (341)
2397. nanciscor (341)
2398. incito (340)
2399. fastigium (340)
2400. sinister (340)
2401. os1 (340)
2402. clam (339)
2403. peric

2851. munimentum (264)
2852. capesso (264)
2853. matutinus (264)
2854. remoueo (264)
2855. rudo (264)
2856. antecedo (264)
2857. munitio (264)
2858. unguentum (264)
2859. gramen (264)
2860. aedilis (264)
2861. adigo (263)
2862. suo (263)
2863. excio (263)
2864. prouideo (263)
2865. juba1 (263)
2866. nutrix (263)
2867. foedus1 (262)
2868. hasdrubal (262)
2869. adipiscor (262)
2870. invitus (262)
2871. praeterquam (262)
2872. catena (261)
2873. colo (261)
2874. milito (261)
2875. incoho (261)
2876. mentio (261)
2877. caligo (261)
2878. pavidus (261)
2879. con-pono (261)
2880. uenerem (260)
2881. exuro (260)
2882. properus (260)
2883. dedico (260)
2884. nubis (260)
2885. qualitas (260)
2886. injuria (260)
2887. idolum (260)
2888. scrutor (259)
2889. paulisper (259)
2890. reverendus (259)
2891. caleo (259)
2892. musa (259)
2893. vestigium (259)
2894. portio (259)
2895. perhibeo (259)
2896. sulpicius (259)
2897. latium (259)
2898. circumspicio (258)
2899. discurro (258)
2900. cliens (258)
2

...

4850. uerber (120)
4851. speculator (120)
4852. jaculus (120)
4853. ilex (120)
4854. duaris (120)
4855. castor (120)
4856. circumuenio (120)
4857. mimus (120)
4858. and (120)
4859. impar (120)
4860. infamis (120)
4861. lucius2 (120)
4862. roseus1 (120)
4863. fessus (120)
4864. sepono (120)
4865. triticum (120)
4866. forsan (120)
4867. decresco (120)
4868. jus (119)
4869. planum (119)
4870. missilis (119)
4871. congero (119)
4872. relatio (119)
4873. famosus (119)
4874. amabilis (119)
4875. antoninus (119)
4876. paucitas (119)
4877. concha (119)
4878. pansa (119)
4879. labrum (119)
4880. affor (119)
4881. iracundus (119)
4882. proconsulis (119)
4883. limus (119)
4884. saeptum (119)
4885. udus (119)
4886. xiii (119)
4887. ausculto (119)
4888. auspico (119)
4889. contradico (119)
4890. religo (119)
4891. verris (119)
4892. rosaceus (119)
4893. sh (119)
4894. qa (119)
4895. ad-sequor (118)
4896. insolentia (118)
4897. resero2 (118)
4898. profundus (118)
4899. astus2 (118)
4900. genital (11

6349. decius (75)
6350. lo (75)
6351. instillo (75)
6352. cichorium (75)
6353. graccho (75)
6354. lon (75)
6355. tor (75)
6356. pharaonis (75)
6357. seruom (75)
6358. stono (75)
6359. ntos (75)
6360. uasto (74)
6361. confingo (74)
6362. deformitas (74)
6363. ultrix (74)
6364. nauigaris (74)
6365. merces1 (74)
6366. industrius (74)
6367. proclivis (74)
6368. providus (74)
6369. quotienscumque (74)
6370. praerumpo (74)
6371. anser1 (74)
6372. intemperantia (74)
6373. sonorus (74)
6374. litigo (74)
6375. europa (74)
6376. phrygiae (74)
6377. uiritim (74)
6378. parthis (74)
6379. faex (74)
6380. nidor (74)
6381. cinna (74)
6382. dependo (74)
6383. ad-tribuo (74)
6384. lustrum (74)
6385. sterilitas (74)
6386. gratiosus (74)
6387. era (74)
6388. depereo (74)
6389. furibundus (74)
6390. relaxo (74)
6391. sanctificatio (74)
6392. congregatio (74)
6393. seruatum (74)
6394. rempublicam (74)
6395. domitius (74)
6396. clodius (74)
6397. pumex (74)
6398. scyphus (74)
6399. triremis (74)
6400. porru

6849. equuleus (66)
6850. indumentum (66)
6851. suscepto (66)
6852. accurro (66)
6853. picenus (66)
6854. alexandriam (66)
6855. mustela (66)
6856. perfuga (66)
6857. aegyptiorum (66)
6858. effreno (66)
6859. auxilia (66)
6860. caesor (66)
6861. injustitia (66)
6862. disjecto (66)
6863. tractas (66)
6864. cibarius (66)
6865. eniteo (66)
6866. dissuadeo (66)
6867. receptus2 (66)
6868. papa (66)
6869. numae (66)
6870. claudii (66)
6871. magicus (66)
6872. magice (66)
6873. obsonium (66)
6874. frigeo (66)
6875. obverso (66)
6876. promereo (66)
6877. inuisus (66)
6878. cupressus (66)
6879. spico (66)
6880. vitupero1 (66)
6881. sponsum (66)
6882. petrum (66)
6883. in-ligo1 (66)
6884. geta (66)
6885. delphis (66)
6886. semis (66)
6887. roscio (66)
6888. epulum (66)
6889. riguus (66)
6890. libycis (66)
6891. aemilium (66)
6892. lxxx (66)
6893. bacchus (66)
6894. antes (66)
6895. de_-rego (66)
6896. cucumis (66)
6897. inmobilis (66)
6898. seq (66)
6899. stoicis (66)
6900. liquesco (66)
6901. p

8348. achaiam (47)
8349. stemma (47)
8350. antidotum (47)
8351. affectus (47)
8352. fronto1 (47)
8353. medimnum (47)
8354. extingui (47)
8355. libani (47)
8356. euocaris (47)
8357. natiuitas (47)
8358. lucanus (47)
8359. expavesco (47)
8360. viola (47)
8361. thymiama (47)
8362. stoicos (47)
8363. abalieno (47)
8364. rosci (47)
8365. sordido (47)
8366. caecinae (47)
8367. bruttiis (47)
8368. origenes (47)
8369. balaam (47)
8370. chaldeorum (47)
8371. zabulon (47)
8372. perqe (47)
8373. lonto (47)
8374. injecto (46)
8375. coluber (46)
8376. indiscretus (46)
8377. onerosus (46)
8378. peruigil (46)
8379. disparus (46)
8380. conclave (46)
8381. primaevus (46)
8382. sublime (46)
8383. planities (46)
8384. spumeus (46)
8385. incessus (46)
8386. reluctor (46)
8387. κατὰ (46)
8388. histrum (46)
8389. mobilitas (46)
8390. gramineus (46)
8391. propugno (46)
8392. justitium (46)
8393. excogitaris (46)
8394. cothurnus (46)
8395. serro (46)
8396. coemo (46)
8397. circumjacio (46)
8398. pango (46)
8

9848. ascensus (34)
9849. divello (34)
9850. consideratio (34)
9851. cursim (34)
9852. democriti (34)
9853. occludo (34)
9854. licenter (34)
9855. subscriptio (34)
9856. incassus (34)
9857. altrinsecus (34)
9858. uehiculum (34)
9859. aceo (34)
9860. distermino (34)
9861. pervicax (34)
9862. brumalis (34)
9863. firmiter (34)
9864. sarmatis (34)
9865. crearis (34)
9866. consternatio (34)
9867. ad-spiro (34)
9868. correctio (34)
9869. coagmento (34)
9870. amissio (34)
9871. cappadociae (34)
9872. incitamentum (34)
9873. ratiocinor (34)
9874. achaiae (34)
9875. navigatio (34)
9876. cyaneus (34)
9877. triuia (34)
9878. exercitium (34)
9879. intemerandus (34)
9880. furiaris (34)
9881. inclinaris (34)
9882. suffigo (34)
9883. pallesco (34)
9884. testaris (34)
9885. apprimus (34)
9886. esaiam (34)
9887. scateo (34)
9888. pala (34)
9889. olympiae (34)
9890. aesopus (34)
9891. ad-commodo2 (34)
9892. libyci (34)
9893. vectigalis (34)
9894. alienaris (34)
9895. christe (34)
9896. manaris (34)
9897